# LLM 評測實戰：用 Twinkle Eval 體驗快速、公平的模型評估流程
<div align="left" style="line-height: 1;">
  <a href="https://discord.gg/Cx737yw4ed" target="_blank" style="margin: 2px;">
    <img alt="Discord" src="https://img.shields.io/badge/Discord-Twinkle%20AI-7289da?logo=discord&logoColor=white&color=7289da" style="display: inline-block; vertical-align: middle;"/>
  </a>
  <a href="https://huggingface.co/twinkle-ai" target="_blank" style="margin: 2px;">
    <img alt="Hugging Face" src="https://img.shields.io/badge/%F0%9F%A4%97%20Hugging%20Face-Twinkle%20AI-ffc107?color=ffc107&logoColor=white" style="display: inline-block; vertical-align: middle;"/>
  </a>
  <a href="https://github.com/ai-twinkle" target="_blank" style="margin: 2px;">
    <img alt="GitHub" src="https://img.shields.io/badge/GitHub-ai--twinkle-181717?logo=github&logoColor=white&color=181717" style="display: inline-block; vertical-align: middle;"/>
  </a>
  <a href="https://colab.research.google.com/github/ai-twinkle/llm-lab/blob/main/courses/2025-0827-llm-eval-with-twinkle/main.ipynb" target="_blank" style="margin: 2px;">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open main.ipynb In Colab" style="display: inline-block; vertical-align: middle;"/>
  </a>
</div>

## 為什麼要對 LLM 做 Benchmark？人工體感不好嗎
> 🤔 **真實案例** <br>
> 客戶：針對這次的模型，我感覺用起來效果沒有很好 <br>
> 訓練師：哪裡不好呢？ <br>
> 客戶：就是回答起來好像都不太準...(loop)

大型語言模型（LLM）應用迅速擴展， 在自然語言理解、生成、程式撰寫及推理等領域展現高度靈活性。然而，若沒有一致且準確的評測標準，很難進行模型間的公平比較，也容易忽視模型真實的性能界限。

評測（Benchmark）提供標準化測試和客觀指標，有助於揭示模型在哪些任務上出色，哪些情況下會失誤，進而成為模型迭代與產品決策的可靠依據。
> 💡 **編按**：假設你的應用場景需要一個在金融領域表現優異的模型，在尚未進行預訓練或微調的情況下，你可以先透過評測方法，客觀挑選在該領域表現較佳的模型。相對地，若你計畫進一步進行預訓練或微調，也應優先選擇基礎能力已經較強的模型，這樣更有助於達到預期的成效。

<p align="center">
  <img src="assets/gpt-oss-120b-mmlu-eval-report.png" width="100%"/><br/>
  <em>圖 1：gpt-oss-120b 在 MMLU 部分子集上的表現成績（圖截置 Twinkle Eval Analyzer）</em>
</p>

在評測過程中，不同子領域的表現差異往往能揭示模型的**強項**與**弱點**。上圖展示了 gpt-oss-120b 在 [MMLU（Massive Multitask Language Understanding）](https://huggingface.co/datasets/cais/mmlu) 中部分子集的表現結果，可以看到模型在「美國外交政策」、「電腦安全」、「專業心理學」等科目有相對較高的正確率（約 0.85–0.87），而在「病毒學」與「全球事實」等領域則略低。

這樣的視覺化結果幫助我們了解：
- 模型的知識覆蓋範圍是否均衡
- 不同領域題目對模型的難度
- 需要額外資料或微調的潛在方向

透過這樣的細部比較，產品團隊或研究人員能根據具體數據來制定改進策略，而不僅僅是依賴整體分數。

<details>
<summary><i>📘 關於 MMLU 資料集的背景介紹</i></summary>

**MMLU（Massive Multitask Language Understanding）** 是由 UC Berkeley 的 Dan Hendrycks 等學者於 2020 年提出，發表於論文  
[Measuring Massive Multitask Language Understanding](https://arxiv.org/abs/2009.03300)。

這個基準測試的設計目的是 **全面衡量大型語言模型的知識廣度與推理能力**。它收錄了 **57 個科目**，範圍涵蓋人文、社會科學、STEM、醫學、法律等領域，題型以選擇題為主。  
MMLU 的特色：
- **跨領域**：內容涵蓋廣泛，適合測試模型的「知識覆蓋面」。
- **難度多樣**：從高中、專業學科到大學水平，能檢驗不同層級的推理與知識應用。
- **已成為業界常用指標**：OpenAI、Anthropic、Google 等多家機構都會引用 MMLU 來評測模型效能

</details>

## 認識 LLM 的「考卷」：常見評測資料集一覽

<p align="center">
  <img src="assets/qwen3-30a3.jpg" width="100%"/><br/>
  <em>圖 2：Qwen3-30A3 在各項評測集上的表現（圖片來源：<a href="https://qwenlm.github.io/blog/qwen3/" target="_blank">Qwen 官方部落格</a>）</em>
</p>

要比較不同大型語言模型的實力，就必須有「共同的考卷」。**評測資料集（Benchmark Dataset）** 就是這張考卷，它們針對不同任務設計題目，讓模型在相同條件下接受測試。

### 國際常用評測集
在大型語言模型的開發與驗證過程中，評測集的選擇至關重要。現有的基準涵蓋多種面向，例如：
- **通用知識與理解**：評估百科知識、推理能力與跨學科理解。
- **數學與邏輯推理**：檢驗模型在計算與邏輯思考上的能力。
- **程式與程式語言理解**：測試模型撰寫、閱讀與執行程式碼的表現。
- **功能與應用能力**：例如函式呼叫、工具使用與實務情境下的任務完成度。

此外，隨著**多模態模型（如圖像、語音、影片）**快速發展，各領域也有對應的專用評測集。然而，本次 Colab 教學著重於**文字（text）模態**，因此以下內容將聚焦於以文字為主要形式的評測資源，並根據不同用途與挑戰類型進行分類。

1. **通用知識與理解類**  
   - [**MMLU**](https://github.com/hendrycks/test) ：涵蓋學科廣、題型多，評估模型的百科知識與推理能力。  
   - [**GPQA**](https://huggingface.co/datasets/Idavidrein/gpqa) ：測試模型對研究級別問題的理解與回答能力。  

2. **數學與邏輯推理類**  
   - [**AIME’24 / AIME’25**](https://artofproblemsolving.com/wiki/index.php/AIME_Problems_and_Solutions) ：專為數學競賽題目設計，檢驗模型的高階數學推理能力。  
   - [**MultiIF**](https://huggingface.co/datasets/THUDM/MultiIF) ：跨多語言的邏輯與推理挑戰。  

3. **程式與程式語言理解類**  
   - [**LiveCodeBench**](https://huggingface.co/datasets/EleutherAI/livecodebench) ：動態程式測試平台，直接測試模型撰寫與執行程式碼的能力。  
   - [**CodeForces Elo**](https://codeforces.com/) ：以線上競賽評分衡量模型的程式解題水準。  

4. **專業與應用領域類**  
   - [**BFCL（Berkeley Function Calling Leaderboard）**](https://huggingface.co/datasets/LLM360/BFCL) ：用來評估模型的函式／工具呼叫能力，包含序列與平行呼叫、跨多種語言與 API 的情境，並以 AST 為基礎做客觀比對；不是多選題知識測驗。  
   - [**LiveBench**](https://huggingface.co/datasets/EleutherAI/livebench) ：為避免測試集汙染而設計的常態更新、多任務基準；題目來自近期資訊來源，能自動、客觀評分（數學、程式、推理、指令遵循、資料分析等），重點是「頻繁更新與可驗證」，不是即時/線上壓力測速。  

5. **複雜綜合評測**  
   - [**ArenaHard**](https://huggingface.co/datasets/lmsys/arena-hard) ：涵蓋多類型題目與挑戰，適合整體性能評估。

> 💡 **編按**：想掌握目前主流的評測資料集，最直接的方法就是關注最新釋出的 LLM 論文與官方部落格，這些內容通常會列出所採用的評測清單與基準。

####  Fun fact: MMLU 可能已經「不夠挑戰性」了？

<p align="center">
  <img src="assets/MMLU-Pro.png" width="100%"/><br/>
  <em>圖 3：TIGER-Lab/MMLU-Pro 資料集概覽</em>
</p>


> 隨著大型語言模型性能的提升，MMLU 的成績普遍趨近飽和，難以有效區分不同模型的推理能力。
> 現在業界反而回頭創建了更具挑戰性的版本，像是更重視深度推理的 **MMLU-Pro**。

<details>
<summary> 了解更多 MMLU-Pro</summary>

**MMLU-Pro** 是由學術界提出，針對 MMLU 進行升級的資料集：  
- 題目更難、更注重推理，選項從原本 4 個增加到 10 個，降低投機成功率。  
- 移除簡單／低品質題目，讓整體測驗更具鑑別力。  
- 模型在 MMLU-Pro 上的平均分數通常比在 MMLU 低 16–33%，且對提示（prompt）的敏感度從 4–5% 降至 2%。  
- 在此 benchmark 上，使用 **Chain-of-Thought (CoT)** 推理方式的模型比直接回答表現更佳──這與原始 MMLU 的情況相反。

進一步更嚴峻的版本是 **MMLU-Pro+** ：它進一步考驗模型是否陷入「捷徑學習」（shortcut learning），增加多答案題型，並新增「正確配對」與「捷徑選擇比例」等評估指標，提升模型判別力與偏見偵測能力。
</details>

### 繁體中文常用評測集
> 💡 **編按**：本 repo 任何提及「繁體中文」即代表**中華民國台灣**所使用的語言，另外也有「正體中文」或「zh-tw」等代表說法。

<p align="center">
  <img src="assets/tmmluplus.png" width="100%"/><br/>
  <em>圖 4：ikala/tmmluplus 資料集概覽</em>
</p>

[**TMMLU+（Traditional Chinese Massive Multitask Language Understanding Plus）**](https://huggingface.co/datasets/ikala/tmmluplus)是一份專為繁體中文語言模型設計的大規模多任務理解評測資料集，具有以下幾個特色：
- **題量與主題涵蓋廣泛**<br>
擁有 **66** 個主題領域，涵蓋從小學、國中、高中到專業階段的知識，包括人文、社會科學與 STEM（如物理、化學、計算機科學）等領域。整體題量超過 **22,000** 筆，比前身 TMMLU 大約 6 倍，主題間分布更為平衡。
- **多選題形式設計**<br>
題型多以 **single-choice** 為主，部分題目可能需要選擇多個選項（例如回答 ABCD），或進行複雜的比較判斷，強化模型的理解與推理能力。
- **Benchmark 基準已提供**<br>
已對多種模型進行測試，包括閉源模型與超過 20 種開放權重的繁中模型，參數規模從 1.8B 到 72B 不等 ￼。測試結果顯示，繁體中文模型普遍落後於簡體模型；即使是大型模型仍顯著不及人類表現（平均約 68%）。
- **設計價值與研發意義**<br>
TMMLU+ 解決了繁體中文基準的不足，特別針對繁簡字形差異與用詞文化差異進行設計，並加入少量開發題目（dev set），方便少樣本學習（few-shot）與提示語設計測試。此外，分析也指出 “fertility score” 是唯一與基準成績高度相關的 Tokenization 度量指標。

<details>
<summary>點擊展開 TMMLU+ 的評測榜（取至官方 dataset card）</summary>
    
| model | STEM | Social Science | Humanities | Other | Average |
|------------|------------|------------|------------|------------|------------|
|Gemini-1.5-pro | 66.18|70.29|61.84|60.30|64.65|
| [Qwen/Qwen-72B](https://huggingface.co/Qwen/Qwen-72B) | 61.12 | 71.65 | 63.00 | 61.31 |64.27|
| gpt-4-0613 | 60.36 | 67.36 | 56.03 | 57.62 |60.34|
| Qwen-max | 59.92 | 66.95 | 57.43 | 56.48 |60.20|
| [Qwen/Qwen-72B-Chat](https://huggingface.co/Qwen/Qwen-72B-Chat) | 55.15 | 66.20 | 55.65 | 57.19 |58.55|
| [Qwen/Qwen-14B](https://huggingface.co/Qwen/Qwen-14B) | 46.94 | 56.69 | 49.43 | 48.81 |50.47|
| Gemini-pro | 45.38 | 57.29 | 48.80 | 48.21 |49.92|
| [01-ai/Yi-34B-Chat](https://huggingface.co/01-ai/Yi-34B-Chat) | 40.24 | 56.77 | 53.99 | 47.58 |49.64|
| Gemini-1.5-flash |53.47|53.42|42.99|46.56|49.11|
| [Reka Flash](https://www.reka.ai/)|45.26|52.91|46.31|43.76|47.06|
| [Qwen/Qwen-14B-Chat](https://huggingface.co/Qwen/Qwen-14B-Chat) | 43.86 | 53.29 | 44.78 | 45.13 |46.77|
| [Qwen/Qwen1.5-14B-Chat](https://huggingface.co/Qwen/Qwen1.5-14B-Chat)|39.65|52.76|43.90|44.95|45.31|
| [01-ai/Yi-6B-Chat](https://huggingface.co/01-ai/Yi-6B-Chat) | 39.62 | 50.24 | 44.44 | 44.26 |44.64|
| Claude-1.3 | 42.65 | 49.33 | 42.16 | 44.14 |44.57|
| [MediaTek-Research/Breeze-7B-Instruct-v0_1](https://huggingface.co/MediaTek-Research/Breeze-7B-Instruct-v0_1)| 36.46 | 48.38 |45.11 |40.75 | 42.67 |
| gpt-3.5-turbo-0613 | 41.56 | 46.72 | 36.73 | 42.03 |41.76|
| [CausalLM/14B](https://huggingface.co/CausalLM/14B) | 39.83 | 44.50 | 39.61 | 41.97 |41.48|
| [Skywork/Skywork-13B-base](https://huggingface.co/Skywork/Skywork-13B-base) | 36.93 | 47.27 | 41.04 | 40.10 |41.33|
| Claude-3-opus |42.95|45.49|35.79|40.24|41.12|
| [Qwen/Qwen-7B](https://huggingface.co/Qwen/Qwen-7B) | 37.53 | 45.48 | 38.09 | 38.96 |40.01|
| [meta-llama/Llama-3-70b-chat-hf](https://docs.together.ai/docs/inference-models) | 34.44 | 47.02 | 37.50 |39.51 | 39.62 | 
| [Qwen/Qwen-7B-Chat](https://huggingface.co/Qwen/Qwen-7B-Chat) | 33.32 | 44.64 | 40.27 | 39.89 |39.53|
| [vivo-ai/BlueLM-7B-Base](https://huggingface.co/vivo-ai/BlueLM-7B-Base) | 33.94 | 41.52 | 37.38 | 38.74 |37.90|
| [baichuan-inc/Baichuan2-13B-Chat](https://huggingface.co/baichuan-inc/Baichuan2-13B-Chat) | 29.64 | 43.73 | 37.36 | 39.88 |37.65|
| [Qwen/Qwen-1_8B](https://huggingface.co/Qwen/Qwen-1_8B) | 32.65 | 38.95 | 38.34 | 35.27 |36.30|
| Claude-2 | 39.65 | 39.09 | 28.59 | 37.47 |36.20|
| [THUDM/chatglm3-6b](https://huggingface.co/THUDM/chatglm3-6b) | 31.05 | 39.31 | 35.64 | 35.60 |35.40|
| [deepseek-ai/deepseek-llm-7b-chat](https://huggingface.co/deepseek-ai/deepseek-llm-7b-chat) | 29.82 | 42.29 | 34.24 | 34.31 |35.17|
| [CausalLM/7B](https://huggingface.co/CausalLM/7B) | 31.03 | 38.17 | 35.87 | 35.39 |35.11|
| [Azure99/blossom-v3_1-mistral-7b](https://huggingface.co/Azure99/blossom-v3_1-mistral-7b) | 32.80 | 36.91 | 32.36 | 34.53 |34.15|
| [google/gemma-7b-it](https://huggingface.co/google/gemma-7b-it) | 31.89 | 35.70 | 34.00 | 33.79 | 33.84 | 
| [Reka Edge](https://www.reka.ai/)|30.02|39.40|31.84|32.36|33.41|
| [microsoft/Orca-2-13b](https://huggingface.co/microsoft/Orca-2-13b) | 24.69 | 39.18 | 33.60 | 31.99 |32.37|
| [Qwen/Qwen-1_8B-Chat](https://huggingface.co/Qwen/Qwen-1_8B-Chat) | 26.60 | 36.36 | 31.81 | 31.96 |31.68|
| [meta-llama/Llama-3-8b-chat-hf](https://docs.together.ai/docs/inference-models) | 31.52 | 34.19 | 28.91 | 31.79 | 31.60 | 
| [TigerResearch/tigerbot-13b-chat-v3](https://huggingface.co/TigerResearch/tigerbot-13b-chat-v3) | 24.73 | 29.63 | 25.72 | 27.22 |26.82|
| [hongyin/mistral-7b-80k](https://huggingface.co/hongyin/mistral-7b-80k) | 24.26 | 23.76 | 22.56 | 24.57 |23.79|
| [deepseek-ai/deepseek-llm-67b-chat](https://huggingface.co/deepseek-ai/deepseek-llm-67b-chat) | 19.10 | 26.06 | 21.51 | 21.77 |22.11|
| [yentinglin/Taiwan-LLM-13B-v2.0-chat](https://huggingface.co/yentinglin/Taiwan-LLM-13B-v2.0-chat) | 18.53 | 27.65 | 17.77 | 21.49 |21.36|
| [GeneZC/MiniChat-3B](https://huggingface.co/GeneZC/MiniChat-3B) | 17.66 | 23.35 | 22.71 | 20.34 |21.02|
| [LinkSoul/Chinese-Llama-2-7b](https://huggingface.co/LinkSoul/Chinese-Llama-2-7b) | 16.55 | 18.39 | 12.97 | 16.13 |16.01|
| [yentinglin/Taiwan-LLM-7B-v2.1-chat](https://huggingface.co/yentinglin/Taiwan-LLM-7B-v2.1-chat) | 14.99 | 16.23 | 15.00 | 16.22 |15.61|
| Claude-instant-1 | 12.52 | 17.13 | 15.10 | 13.57 |14.58|
| [FlagAlpha/Atom-7B](https://huggingface.co/FlagAlpha/Atom-7B) | 5.60 | 13.57 | 7.71 | 11.84 |9.68|

</details>

## 評測工具與框架概覽

除了設計評測指標外，模型開發過程中也需要合適的工具與框架來支援測試與驗證。這類工具能協助我們更快速地管理資料集、執行評測流程、紀錄結果，甚至擴充到更複雜的功能，例如函式呼叫或多任務情境。以下將概覽目前常見的評測框架。

| 框架 / 工具 | 功能與特色 | 適用場景 |
|-------------|------------|-----------|
| **EleutherAI / lm-evaluation-harness** | 一個統一評測框架，支持超過 60 個標準 benchmark，涵蓋 hundreds of subtasks。支援多種模型載入方式（Transformers、本地模型、OpenAI API 等），並支援自訂 prompt 與 metrics。廣泛被研究社群與 Hugging Face Leaderboard 採用。 | 快速、 reproducible 地在學術 benchmark 上測試模型 |
| **Hugging Face / LightEval** | 一體化 Toolkit，支持多種後端（transformers、vLLM、TGI、Nanotron 等），並可儲存 sample-by-sample 結果、上傳 Hf Hub 或 S3。延續自 lm-harness，強調速度、靈活度與可客製化任務。 | 適合在 Hugging Face 生態系內進行快速、彈性評測 |
| **Berkeley Function Calling Leaderboard (BFCL)** | 專為評測 LLM 的 function／tool calling 設計的 benchmark。涵蓋單回合、多語言、並行、多階段 agentic 使用場景，並以 AST 結構匹配代替實際執行進行評分。適合評估 LLM 在工具使用與 agentic 推理上的能力。 | 測試模型在 real-world API 呼叫、狀態管理、多步推理上的能力 |
| **DeepEval** | 類似 Pytest 的 LLM 評測框架，提供 >14 項研究支持的 metric（如 G-Eval、hallucination、faithfulness、RAG 特有評分、agentic task 完成度等），支援 synthetic data、CI/CD 整合與 Red Team 安全測試。 | 適合研發流程中進行多維度、自動化測試與安全監控。 |
| **其他相關框架** | - **UltraEval**：輕量化、模組化的評測平台，方便混搭不同模型、任務、評分指標。 <br> - **ComplexFuncBench**：功能呼叫領域的長 context、多步參數推理 benchmark，並附 eval pipeline“ComplexEval”。 | - 適合柔性、高度自訂的研究用途 <br> - 用於檢驗模型在複雜呼叫與長依賴場景下的表現 |

### Twinkle Eval

<p align="center">
  <img src="assets/twinkle-eval.png" width="100%"/><br/>
  <em>圖 3：Twinkle Eval</em>
</p>

[Twinkle Eval](https://github.com/ai-twinkle/Eval) 是 Twinkle AI 在開發推理模型過程中自行開發的一套評測工具。當時我們面臨推理過程**耗時**且**缺乏能夠詳細紀錄每題結果**的評測工具，因此決定自行研發。Twinkle Eval 能夠記錄每題的完整結果，並支援**隨機排列題目順序**，使評測過程更加公平。目前該工具主要應用於選擇題部分。

<p align="center">
  <img src="assets/tmmlu_eval_time_rounded_seconds.png" width="100%"/><br/>
  <em>圖 5：Twinkle Eval 在不同模型大小與任務類型下皆能顯著提升效能，最高達近 17 倍速度優勢，同時保持準確率一致</em>
</p>

從上圖可以看見，不論是小型還是大型參數量的模型，Twinkle Eval 都能在推理評測上展現顯著的效率提升。對於不同任務類型與題目配置，它不僅能將評測速度提升至原本的數倍，甚至在部分情境下達到近 17 倍的加速效益，同時仍維持與原始流程相當的一致性與準確率。

特別是在像 [ikala/tmmlu+](https://huggingface.co/datasets/ikala/tmmluplus) 這類動輒破萬筆的資料集上，評測效率更是關鍵。若沒有高速的評測工具，模型開發將受到龐大資料量拖累，導致訓練與調整週期過長。Twinkle Eval 能讓這些大量樣本的評測在可接受的時間內完成，確保訓練能快速迭代、縮短開發時程。

#### 下載 Twinkle Eval

In [16]:
!pip install git+https://github.com/ai-twinkle/Eval.git pyyaml
import twinkle_eval, sys
print("Twinkle Eval version:", twinkle_eval.__version__)

  Cloning https://github.com/ai-twinkle/Eval.git to /tmp/pip-req-build-pumqe9f9
  Running command git clone --filter=blob:none --quiet https://github.com/ai-twinkle/Eval.git /tmp/pip-req-build-pumqe9f9
  Resolved https://github.com/ai-twinkle/Eval.git to commit ae71a6981499a9c35b611cc603423ff637cad087
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Twinkle Eval version: 1.1.0


#### 以小型評測集 Formosa Bench 進行初步測試

<p align="center">
  <img src="assets/Formosa-bench.png" width="100%"/><br/>
  <em>圖 6：福爾摩沙資料集（Formosa Bench）收錄了大量與台灣社會、歷史、地理、政府及人文相關的問答，旨在測試語言模型對中華民國台灣人文社會知識的理解程度。</em>
</p>

與其他繁體中文資料集不同，[**Formosa Bench**](https://huggingface.co/datasets/lianghsun/Formosa-bench) 並非以國家考試題目為主，而是從中華民國台灣的日常介紹、文化特色與常識中，精選出能反映這片土地特質的問答內容。這樣的設計更能檢驗模型對中華民國台灣的理解深度。

在本次示範中，我們以這個**小型資料集**作為範例，進行測試與驗證，展示 Twinkle Eval 的完整評測流程。

<p align="center">
  <img src="assets/Formosa-bench-example.png" width="100%"/><br/>
  <em>圖 7：Formosa Bench 子集示例，題目以選擇題形式呈現</em>
</p>

In [22]:
from datasets import load_dataset
import pandas as pd
from pathlib import Path

# 1. 載入完整的 history split
ds = load_dataset("lianghsun/Formosa-bench", "history", split="test")

# 2. 確認欄位，做基本清理
required_cols = {"question", "A", "B", "C", "D", "answer"}
missing = required_cols - set(ds.column_names)
assert not missing, f"缺少欄位：{missing}；實際欄位：{ds.column_names}"

df = ds.to_pandas()[["question", "A", "B", "C", "D", "answer"]].copy()
df["answer"] = df["answer"].astype(str).str.strip().str.upper()
df = df[df["answer"].isin(list("ABCD"))]

# 3. 存放到 datasets/ 資料夾
datasets_dir = Path("datasets")
datasets_dir.mkdir(exist_ok=True)
out_path = datasets_dir / "formosa_bench_history.jsonl"
df.to_json(out_path, orient="records", lines=True, force_ascii=False)
print(f"✅ 已存檔：{out_path}，共 {len(df)} 筆")

# 4. 預覽
df.head(10)

✅ 已存檔：datasets/formosa_bench_history.jsonl，共 23 筆


,question,A,B,C,D,answer
0,八田與一是臺灣最重要且偉大的土木工程技師之一。他建造哪個水庫與嘉南大圳，發展農業灌溉水路系統...,曾文水庫,翡翠水庫,南化水庫,烏山頭水庫,D
1,下列何者是明代以後，漢人曾用以稱呼台灣全島或部分台灣的名稱？,東番,大員,瑠求,毗舍耶,C
2,下列對於臺灣發展史上的相關敘述，何項錯誤？,劉銘傳結束明鄭政權，促成臺灣納入清朝版圖,唐末已有漢人在澎湖活動,元朝曾在澎湖設巡檢司,臺灣原住民族屬南島語系,A
3,下列臺灣古今地名的對照，何者錯誤？,雞籠―基隆市,透水─秀水鄉,霧社─埔里鄉,打狗─高雄市,C
4,下列臺灣古蹟中，何者是最早興建？,赤崁樓,臺灣總督府,淡水紅毛城,打狗英國領事館,C
5,中華民國史上首位女性國家元首是誰？,王清峰,蔡英文,呂秀蓮,蕭美琴,B
6,中華民國史上首位無黨籍的副總統是誰？,朱惠良,蕭萬長,陳建仁,王清峰,C
7,中華民國行憲後的第一任總統是誰？,孫中山,蔣中正,袁世凱,李宗仁,B
8,中華民國制定《中華民國憲法》是在何時？,民國35年,民國36年,民國37年,民國38年,A
9,中華民國於何時正式宣告成立？,1911年10月10日,1912年1月1日,1928年6月,1931年9月,B


In [19]:
!twinkle-eval --init

✅ 配置檔案已創建: config.yaml

📝 接下來請編輯配置檔案，設定：
  1. LLM API 設定 (base_url, api_key)
  2. 模型名稱 (model.name)
  3. 資料集路徑 (evaluation.dataset_paths)

💡 編輯完成後，使用以下命令開始評測：
   twinkle-eval --config config.yaml


#### 設定你的 LLM API 以及評測集路徑

在專案資料夾中有一份 config.yaml 設定檔，你需要在其中填寫 LLM API 的連線資訊，包括 base_url、api_key 以及 name：
```yaml
llm_api:
  base_url: "" # API 伺服器網址
  api_key: ""  # API 金鑰
  ...
model:
  name: ""     # API 名稱
```
此外，還需指定評測資料集的路徑，可在同一份設定檔中進行設定：
```yaml
evaluation:
  dataset_paths: # 資料集路徑
    - "datasets/"
```

完成設定後，就可以直接透過以下指令啟動評測流程：

In [27]:
!twinkle-eval --config config.yaml

掃描目錄： datasets/
正在讀取: datasets/formosa_bench_history.jsonl
處理回應中: 100%|██████████████████████████████| 23/23 [00:00<00:00, 183.62it/s]
✅ 評測完成，結果已儲存至 results/eval_results_20250825_0845_run0.jsonl
正在讀取: datasets/formosa_bench_history.jsonl
處理回應中: 100%|██████████████████████████████| 23/23 [00:00<00:00, 171.32it/s]
✅ 評測完成，結果已儲存至 results/eval_results_20250825_0845_run1.jsonl
正在讀取: datasets/formosa_bench_history.jsonl
處理回應中: 100%|██████████████████████████████| 23/23 [00:00<00:00, 178.41it/s]
✅ 評測完成，結果已儲存至 results/eval_results_20250825_0845_run2.jsonl
正在讀取: datasets/formosa_bench_history.jsonl
處理回應中: 100%|██████████████████████████████| 23/23 [00:00<00:00, 185.69it/s]
✅ 評測完成，結果已儲存至 results/eval_results_20250825_0845_run3.jsonl
正在讀取: datasets/formosa_bench_history.jsonl
處理回應中: 100%|██████████████████████████████| 23/23 [00:00<00:00, 725.78it/s]
✅ 評測完成，結果已儲存至 results/eval_results_20250825_0845_run4.jsonl
已執行 100.0% (1/1) 
資料集 datasets/ 評測完成，平均正確率: 43.48% (±4.76%)


在這次的示範中，我們以 Formosa Bench 為例，系統會先掃描 `datasets/` 目錄並讀取 `formosa_bench_history.jsonl`，接著進行多次評測。由於在 config.yaml 中事先設定了：
```yaml
repeat_runs: 5
```
因此系統會重複執行五次，其中**處理題庫中**表示正在對題目與選項進行隨機化處理，**處理回應中**則代表透過 LLM API 執行模型評測。

所有結果會自動儲存為 `JSONL` 格式，例如：
- `results/eval_results_20250825_0845_run0.jsonl`
- `results/eval_results_20250825_0845_run1.jsonl`
- ...
- `results/eval_results_20250825_0845_run5.jsonl`

最後，系統會彙整五輪結果並計算整體表現。

#### 看看評測結果

In [30]:
!ls results

eval_results_20250825_0845_run0.jsonl  eval_results_20250825_0845_run3.jsonl
eval_results_20250825_0845_run1.jsonl  eval_results_20250825_0845_run4.jsonl
eval_results_20250825_0845_run2.jsonl  results_20250825_0845.json


`eval_results_<datetime>_run#.jsonl` 是用紀錄每一輪執行時，LLM 回答每一筆的內容
```json
{"question_id": 12, "question": "元朝開始在臺灣何地設置巡檢司？\nA: 苗栗\nB: 澎湖\nC: 臺南\nD: 基隆", "correct_answer": "B", "llm_output": "\\box{臺南}", "llm_reasoning_output": null, "predicted_answer": null, "is_correct": false, "usage_completion_tokens": 7, "usage_prompt_tokens": 146, "usage_total_tokens": 153}
```

- **question_id**：題目在資料集中的唯一識別碼。
- **question**：題目本文與選項（A／B／C／D）。
- **correct_answer**：正解（以選項代號表示，如 "B"）。
- **llm_output**：模型原始輸出（未經我們解析或正規化）。
- **llm_reasoning_output**：若模型有額外輸出思考／解題步驟，會存於此；本例為 null。
- **predicted_answer**：Twinkle Eval 從 llm_output 解析出的最終作答（應為 A/B/C/D 其中之一）。若解析失敗則為 null。
- **is_correct**：是否答對（true/false）。
- **usage_prompt_tokens** / **usage_completion_tokens** / **usage_total_tokens**：本題推論的 Token 使用量統計（提示／完成／合計），便於後續成本與效能分析。

#### 為什麼這題被判錯？

本次的 system prompt（摘要如下）規定必須以 `\box{選項}` 的格式輸出選項代號（A/B/C/D）：
```markdown
使用者將提供一個題目，並附上選項 A、B、C、D
請…根據題意選出最符合的選項，並將選項以以下格式輸出：
\box{選項}
請確保僅將選項包含在 { } 中…
```

但 `llm_output` 為 `\box{臺南}`，也就是把選項文字放進框，而非選項代號（應該是 `\box{C}` 或 `\box{B}` 等）。因此解析器無法抽取合法選項，導致 `predicted_answer` = `null`，進而一律視為錯誤（`is_correct` = `false`）。

`results_<dateitme>.json` 則是紀錄這整次執行評測的統計數字
```json
{
    "timestamp": "20250825_0845",
    "config": {
        "llm_api": {
            "base_url": "https://litellm-ekkks8gsocw.dgx-coolify.apmic.ai",
            "disable_ssl_verify": false,
            "api_rate_limit": 2,
            "max_retries": 5,
            "timeout": 600,
            "type": "openai"
        },
        "model": {
            "name": "gemma-3-12b-it",
            "temperature": 0.0,
            "top_p": 0.9,
            "max_tokens": 4096,
            "frequency_penalty": 0.0,
            "presence_penalty": 0.0,
            "extra_body": null
        },
        "evaluation": {
            "dataset_paths": [
                "datasets/"
            ],
            "evaluation_method": "box",
            ...(省略)...

    "dataset_results": {
        "datasets/": {
            "results": [
                {
                    "file": "datasets/formosa_bench_history.jsonl",
                    "accuracy_mean": 0.43478260869565216,
                    "accuracy_std": 0.047628048478710085,
                    "individual_runs": {
                        "accuracies": [
                            0.5217391304347826,
                            0.43478260869565216,
                            0.43478260869565216,
                            0.391304347826087,
                            0.391304347826087
                        ],
                        "results": [
                            "results/eval_results_20250825_0845_run0.jsonl",
                            "results/eval_results_20250825_0845_run1.jsonl",
                            "results/eval_results_20250825_0845_run2.jsonl",
                            "results/eval_results_20250825_0845_run3.jsonl",
                            "results/eval_results_20250825_0845_run4.jsonl"
                        ]
                    }
                }
            ],
            "average_accuracy": 0.43478260869565216,
            "average_std": 0.047628048478710085
        }
    },
    "duration_seconds": 57.069114  
```

上述的主要欄位說明

- **timestamp**  
  這次評測的時間戳記，方便追蹤不同執行批次，例如 `20250825_0845`。

- **config**  
  儲存這次評測時的完整設定：  
  - **llm_api**：API 相關設定（伺服器網址、類型、限速、重試次數、逾時時間等）。  
  - **model**：模型設定（模型名稱、生成參數 `temperature`、`top_p`、`max_tokens` 等）。  
  - **evaluation**：評測設定（資料集路徑、使用的評測方法 `box`、重複執行次數等）。  

- **dataset_results**  
  存放每個資料集的評測結果：  
  - **results**：列出所有被評測的檔案與其表現。  
    - **file**：資料集檔名，例如 `formosa_bench_history.jsonl`。  
    - **accuracy_mean / accuracy_std**：這個資料集的平均正確率與標準差。  
    - **individual_runs**：紀錄每次執行的細節：  
      - **accuracies**：每一輪執行的正確率清單。  
      - **results**：對應每輪的結果檔路徑。  
  - **average_accuracy / average_std**：所有資料集的整體平均表現。  

- **duration_seconds**：這次評測的總耗時（單位：秒）。

### 視覺化報告（Visualized Report）

<p align="center">
  <img src="assets/twinkle-eval-anlyzer.png" width="100%"/><br/>
  <em>圖 8：Twinkle Eval Analyzer 介面，提供互動式檢視與結果統計。</em>
</p>

完整的互動式評測結果可於此[連結](https://a6b98eaff3dc.ngrok-free.app/)查看。

## Quiz: 請用 Twinkle Eval 執行 ikala/tmmluplus 評測集

請針對社群給的 LLM API 資源，透過 **Twinkle Eval** 去評測模型，你也可以選擇任一個子集合上面進行測試，本例建議試著從 terminal 執行。

In [28]:
# Hint：請先開啟 terminal，再下載 tmmluplus 評測集，然後再執行命令
!git clone https://huggingface.co/datasets/ikala/tmmluplus